In [3]:
# test cifar, imagenet acc

from Dataloaders import dataloader_image_20
train, test, val = dataloader_image_20.Dataloader_imagenet_20_integrated()

from Models import mobilenetv2, resnet
# client, server = mobilenetv2.mobilenetv2_splitter(num_classes=1000, weight_root='./Weights/imagenet/')
client, server = resnet.resnet_splitter(num_classes=1000, weight_root='./Weights/imagenet/', layers=50)

client = client.cuda()
server = server.cuda()
client.eval()
server.eval()

import torch
from tqdm import tqdm

# calculate the correct rate of each label
with torch.no_grad():
    acc = 0
    label_correct_rate = [0] * 20
    hit_rate = [0] * 20
    target_list = [109, 618, 536, 546, 347, 40, 415, 619, 208, 314, 8, 525, 714, 52, 221, 523, 785, 951, 984, 965]
    for i, (data, target) in enumerate(tqdm(test)):
        data, target = data.cuda(), target.cuda()
        
        output = client(data)
        output = server(output)
        output = output.argmax(dim=1)
        acc += (output == target).sum().item()
        # for i in target.cuda().cpu().numpy():
        #     if i not in target_list:
        #         target_list.append(i)
        for i in range(output.size(0)):
            for j in range(20):
                if target[i] == target_list[j]:
                    hit_rate[j] += 1
                    if output[i] == target[i]:
                        label_correct_rate[j] += 1

    print(len(test.dataset))
    print(acc / len(test.dataset))
    print([label_correct_rate[i] / max(1,hit_rate[i]) for i in range(20)])
    print(hit_rate)
    # print(target_list)

  0%|          | 0/51 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 3, got 2)